<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch07/ch07-02-blending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    import sys
    import os

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git

        sys.path.append('/content/kagglebook/ch07')
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場
        sys.path.append(os.path.join(colab_dir, 'ch07'))

ON_COLAB: True
USE_GIT: True
fatal: destination path 'kagglebook' already exists and is not an empty directory.


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [4]:
# neural net用のデータ
if ON_COLAB:
    if USE_GIT:
        train_nn = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed_onehot.csv')
    else:
        train_nn = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed_onehot.csv'))
else:
    train_nn = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x_nn = train_nn.drop(['target'], axis=1)
train_y_nn = train_nn['target']
if ON_COLAB:
    if USE_GIT:
        test_x_nn = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed_onehot.csv')
    else:
        test_x_nn = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed_onehot.csv'))
else:
    test_x_nn = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [5]:
# ---------------------------------
# hold-outデータへの予測値を用いたアンサンブル
# ----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [6]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_index = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_index]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_index]
tr_x_nn, va_x_nn = train_x_nn.iloc[tr_idx], train_x_nn.iloc[va_index]

In [7]:
# models.pyにModel1_1, Model1_2, Model2を定義しているものとする
# 各クラスは、fitで学習し、predictで予測値の確率を出力する
from models import Model1Xgb, Model1NN, Model2Linear

In [8]:
# 1層目のモデル
# 学習データで学習し、hold-outデータとテストデータへの予測値を出力する
model_1a = Model1Xgb()
model_1a.fit(tr_x, tr_y, va_x, va_y)
va_pred_1a = model_1a.predict(va_x)
test_pred_1a = model_1a.predict(test_x)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:26:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-logloss:0.41663	eval-logloss:0.43550
[1]	train-logloss:0.37126	eval-logloss:0.39889
[2]	train-logloss:0.33889	eval-logloss:0.37205
[3]	train-logloss:0.31320	eval-logloss:0.35606
[4]	train-logloss:0.29062	eval-logloss:0.33709
[5]	train-logloss:0.27315	eval-logloss:0.32549
[6]	train-logloss:0.25735	eval-logloss:0.31328
[7]	train-logloss:0.24299	eval-logloss:0.30434
[8]	train-logloss:0.22850	eval-logloss:0.29755
[9]	train-logloss:0.21984	eval-logloss:0.29490


In [9]:
model_1b = Model1NN()
model_1b.fit(tr_x_nn, tr_y, va_x_nn, va_y)
va_pred_1b = model_1b.predict(va_x_nn)
test_pred_1b = model_1b.predict(test_x_nn)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 0.4937 - val_loss: 0.3844
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.3597 - val_loss: 0.3737
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.3344 - val_loss: 0.3691
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.3141 - val_loss: 0.3713
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.3015 - val_loss: 0.3525
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.2764 - val_loss: 0.3355
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.2401 - val_loss: 0.3225
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.2195 - val_loss: 0.3000
Epoch 9/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1975 - val_loss: 0.2861
Epoch 10/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1794 - val_loss: 0.2943


AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [ ]:
# hold-outデータでの精度を評価する
print(f'logloss: {log_loss(va_y, va_pred_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(va_y, va_pred_1b, eps=1e-7):.4f}')

In [ ]:
# hold-outデータとテストデータへの予測値を特徴量としてデータフレームを作成
va_x_2 = pd.DataFrame({'pred_1a': va_pred_1a, 'pred_1b': va_pred_1b})
test_x_2 = pd.DataFrame({'pred_1a': test_pred_1a, 'pred_1b': test_pred_1b})

In [ ]:
# 2層目のモデル
# Hold-outデータすべてで学習しているので、評価することができない。
# 評価を行うには、Hold-outデータをさらにクロスバリデーションする方法が考えられる。
model2 = Model2Linear()
model2.fit(va_x_2, va_y, None, None)
pred_test_2 = model2.predict(test_x_2)